In [ ]:
# 과제 3

In [ ]:
# 제출 안내
# • 1번, 2번, 3번 내용 레포트 작성하여 제출 할 것
# § 각 결과에 대한 분석 의견 작성 필수
# § 소스코드 레포트 뒷부분에 첨부
# • 제출 마감 : 4월 15일
# • 제출 방식 : CTL 과제 게시판 제출

In [ ]:
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import scipy
import sympy
import datetime

In [ ]:
## open cv 설치
!pip install opencv-contrib-python

In [ ]:
import cv2
cv2.__version__

In [ ]:
from google.colab.patches import cv2_imshow
from google.colab import drive
import cv2
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import cv2
import numpy as np

# 과제 3

# Image Denoising
# • Gaussian noisy image 및 Salt-pepper noisy image 활용

path1 = '/content/gdrive/MyDrive/차량비전시스템/VisionSystem/Assignment3_Image/Copy of Lena-Gaussian-noise2 (1).jpg'
image1=cv2.imread(path1,cv2.IMREAD_COLOR)

path2 = '/content/gdrive/MyDrive/차량비전시스템/VisionSystem/Assignment3_Image/fce(salt_pepper noise).bmp'
image2=cv2.imread(path2,cv2.IMREAD_COLOR)


# 1. 수행 내용 및 제출물 (두 개의 이미지 모두에 대해)
# *단, Gaussian filter 의 분산값은 1.0으로 설정
print("1. 수행 내용 및 제출물 (두 개의 이미지 모두에 대해)")
print("*단, Gaussian filter 의 분산값은 1.0으로 설정")

# 노이즈 제거 함수
def denoise_image(image, filter_size):
    # Average filter
    average = cv2.blur(image, (filter_size, filter_size))

    # Median filter
    median = cv2.medianBlur(image, filter_size)

    # Gaussian filter
    gaussian = cv2.GaussianBlur(image, (filter_size, filter_size), sigmaX=1.0)

    return average, median, gaussian

# 필터 사이즈
filter_sizes = [3, 7, 11, 17]

# 필터 적용 및 결과물 출력
for i, img in enumerate([image1, image2]):
    for size in filter_sizes:
        print(f"필터의 사이즈를 {size} by {size}으로 설정 후 노이즈 제거 결과")
        average, median, gaussian = denoise_image(img, size)

         # 결과 이미지 출력
        print("<Gaussian noisy image>" if i == 0 else "<Salt-pepper noisy image>")
        print("§ Average filter 활용한 노이즈 제거")
        cv2_imshow(np.hstack([img, average]))
        print("§ Median filter 활용한 노이즈 제거")
        cv2_imshow(np.hstack([img, median]))
        print("§ Gaussian filter 활용한 노이즈 제거")
        cv2_imshow(np.hstack([img, gaussian]))
        print()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 2. 수행 내용 및 제출물 (두 개의 이미지 모두에 대해, 필터의
# 사이즈는 5 by 5로 고정)
print("2. 수행 내용 및 제출물 (두 개의 이미지 모두에 대해, 필터의 사이즈는 5 by 5로 고정)")

# 필터 적용 함수
def apply_filters(image, filter_size, variance):

    # 분산값을 표준 편차로 변환
    sigma = np.sqrt(variance)

    # Average filter
    average = cv2.blur(image, (filter_size, filter_size))

    # Median filter
    median = cv2.medianBlur(image, filter_size)

    # Gaussian filter
    gaussian = cv2.GaussianBlur(image, (filter_size, filter_size), sigmaX=sigma)

    return average, median, gaussian

# 필터 크기와 분산 설정
filter_size = 5
variances = [2.0, 4.0, 7.0, 10.0]

# 이미지에 대해 각 필터 적용
for i, img in enumerate([image1, image2]):
    for variance in variances:
        average, median, gaussian = apply_filters(img, filter_size, variance)

        # 결과 이미지 출력
        print("<Gaussian noisy image>" if i == 0 else "<Salt-pepper noisy image>")
        print("§ Average filter 적용 결과")
        cv2_imshow(np.hstack([img, average]))
        print("§ Median filter 적용 결과")
        cv2_imshow(np.hstack([img, median]))
        print("§ Gaussian filter (분산={}) 적용 결과".format(variance))
        cv2_imshow(np.hstack([img, gaussian]))
        print()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Edge Detection
# • 과제 수행에 사용할 이미지 선택
# § 2개 이상 선택
# § 최소 1개는 노이즈가 첨가된 이미지이어야 함

path1 = '/content/gdrive/MyDrive/차량비전시스템/VisionSystem/Assignment3_Image/Copy of Lena-Gaussian-noise2 (1).jpg'
path2 = '/content/gdrive/MyDrive/차량비전시스템/VisionSystem/Assignment3_Image/fce(salt_pepper noise).bmp'
image1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)

# • Edge detection

# 엣지 검출 함수
def apply_edge_detection(image):
    # § 로버트 크로스
    kernel_roberts_x = np.array([[1, 0], [0, -1]])
    roberts_x = cv2.filter2D(image, cv2.CV_64F, kernel_roberts_x)
    kernel_roberts_y = np.array([[0, 1], [-1, 0]])
    roberts_y = cv2.filter2D(image, cv2.CV_64F, kernel_roberts_y)
    roberts_cross = cv2.magnitude(roberts_x, roberts_y)

    # § 소벨
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)

    # § 프리윗
    kernel_prewitt_x = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
    kernel_prewitt_y = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    prewitt_x = cv2.filter2D(image, cv2.CV_64F, kernel_prewitt_x)
    prewitt_y = cv2.filter2D(image, cv2.CV_64F, kernel_prewitt_y)
    prewitt = cv2.magnitude(prewitt_x, prewitt_y)

    # § 캐니
    canny = cv2.Canny(image, 100, 200)

    # § 라플라시안
    laplacian = cv2.Laplacian(image, cv2.CV_64F)

    # § LoG
    gaussian = cv2.GaussianBlur(image, (3, 3), 0)
    laplacian_of_gaussian = cv2.Laplacian(gaussian, cv2.CV_64F)

    return roberts_cross, sobel, prewitt, canny, laplacian, laplacian_of_gaussian


# 3. 제출물
# § 위 6개의 필터를 본인이 선택한 이미지에 모두 적용한 결과
print("3. 제출물")
print("§ 위 6개의 필터를 본인이 선택한 이미지에 모두 적용한 결과")

# 엣지 검출 적용
roberts_cross1, sobel1, prewitt1, canny1, laplacian1, laplacian_of_gaussian1 = apply_edge_detection(image1)
roberts_cross2, sobel2, prewitt2, canny2, laplacian2, laplacian_of_gaussian2 = apply_edge_detection(image2)

# 결과 출력
cv2_imshow(np.hstack([image1, roberts_cross1, sobel1, prewitt1, canny1, laplacian1, laplacian_of_gaussian1]))
cv2_imshow(np.hstack([image2, roberts_cross2, sobel2, prewitt2, canny2, laplacian2, laplacian_of_gaussian2]))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Edge Detection - 코드 수정.ver
# • 과제 수행에 사용할 이미지 선택
# § 2개 이상 선택
# § 최소 1개는 노이즈가 첨가된 이미지이어야 함

import cv2
import numpy as np
from matplotlib import pyplot as plt

path1 = '/content/gdrive/MyDrive/차량비전시스템/VisionSystem/Assignment3_Image/Copy of Lena-Gaussian-noise2 (1).jpg'
path2 = '/content/gdrive/MyDrive/차량비전시스템/VisionSystem/Assignment3_Image/fce(salt_pepper noise).bmp'
image1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)

# • Edge detection

# 엣지 검출 함수
def roberts_cross_func(image):
    # § 로버트 크로스
    kernel_roberts_x = np.array([[1, 0], [0, -1]])
    roberts_x = cv2.filter2D(image, cv2.CV_64F, kernel_roberts_x)
    kernel_roberts_y = np.array([[0, 1], [-1, 0]])
    roberts_y = cv2.filter2D(image, cv2.CV_64F, kernel_roberts_y)
    roberts_cross = cv2.magnitude(roberts_x, roberts_y)
    return roberts_cross

def sobel_func(image):
    # § 소벨
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    sobel = cv2.magnitude(sobel_x, sobel_y)
    return sobel

def prewitt_func(image):
    # § 프리윗
    kernel_prewitt_x = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
    kernel_prewitt_y = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    prewitt_x = cv2.filter2D(image, cv2.CV_64F, kernel_prewitt_x)
    prewitt_y = cv2.filter2D(image, cv2.CV_64F, kernel_prewitt_y)
    prewitt = cv2.magnitude(prewitt_x, prewitt_y)
    return prewitt

def canny_func(image):
    # § 캐니
    canny = cv2.Canny(image, 100, 200)
    return canny

def laplacian(image):
    # § 라플라시안
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    return laplacian

def log_func(image):
    # § LoG
    gaussian = cv2.GaussianBlur(image, (3, 3), 0)
    laplacian_of_gaussian = cv2.Laplacian(gaussian, cv2.CV_64F)
    return laplacian_of_gaussian

# 3. 제출물
# § 위 6개의 필터를 본인이 선택한 이미지에 모두 적용한 결과
print("3. 제출물")
print("§ 위 6개의 필터를 본인이 선택한 이미지에 모두 적용한 결과")

# 엣지 검출 적용 및 결과 출력
filters = [roberts_cross_func, sobel_func, prewitt_func, canny_func, laplacian, log_func]
titles = ['Roberts Cross', 'Sobel', 'Prewitt', 'Canny', 'Laplacian', 'LoG']

for title, filter_func in zip(titles, filters):
    plt.figure(figsize=(20, 5))
    result1 = filter_func(image1)
    plt.subplot(1, 2, 1)
    plt.imshow(result1, cmap='gray')
    plt.title(title + ' Image1')
    plt.axis('off')

    result2 = filter_func(image2)
    plt.subplot(1, 2, 2)
    plt.imshow(result2, cmap='gray')
    plt.title(title + ' Image2')
    plt.axis('off')

    plt.show()

Output hidden; open in https://colab.research.google.com to view.